In [ ]:
import pandas as pd
import networkx as nx
import os
import unicodedata
import math
import pickle
import numpy as np

import psycopg2
import yaml

from nltk.tokenize import WordPunctTokenizer

from collections import defaultdict

### Lista de Problemas

Nome repetido
Acentuação
Nome composto +3
Capitalização
Tempo computacional

In [ ]:
path = 'congresso_em_numeros/'
deputados = 'deputados.csv'
tramitacao = 'tramitacao.csv'

In [ ]:
def connect_sqlalchemy():
    with open('server_config.yaml', 'r') as f:
        server = yaml.load(f)

    host = server['host']
    database = server['database']
    user = server['user']
    password = server['password']

    from sqlalchemy import create_engine
    url = 'postgresql://{}:{}@{}/{}'
    url = url.format(user, password, host, database)
    return create_engine(url)

In [ ]:
con = connect_sqlalchemy()

In [ ]:
dados_tramitacao = pd.read_sql_table(table_name='camdep_proposicoes_tramitacao', schema='c_camdep', con=con)

In [ ]:
dados_deputado = pd.read_csv(os.path.join(path,deputados), names = ['nome_completo', 'nome', 'id'])
#dados_tramitacao = pd.read_csv(os.path.join(path,tramitacao))

Removendo NA

In [ ]:
dados_tramitacao = dados_tramitacao.dropna(axis = 0)


Removendo pontuação

In [ ]:
def remove_accents(input_str):
    if type(input_str) is float:
        if math.isnan(input_str):
            return 

    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [ ]:
dados_tramitacao["despacho"] = dados_tramitacao["despacho"].apply(lambda x: remove_accents(x))
dados_deputado["nome_completo"] = dados_deputado["nome_completo"].apply(lambda x: remove_accents(x))
dados_deputado["nome"] = dados_deputado["nome"].apply(lambda x: remove_accents(x))

Deixando tudo em caixa baixa

In [ ]:
dados_tramitacao["despacho"] = dados_tramitacao["despacho"].apply(lambda x: x.lower())
dados_deputado["nome_completo"] = dados_deputado["nome_completo"].apply(lambda x: x.lower())
dados_deputado["nome"] = dados_deputado["nome"].apply(lambda x: x.lower())

In [ ]:
dados_deputado = dados_deputado.drop_duplicates(['nome', 'id'])

Modificando as ids que não deviam estar duplicadas

In [ ]:
import hashlib

In [ ]:
D = defaultdict(list)
for i,item in enumerate(dados_tramitacao.id_tramitacao):
    D[item].append(i)
D = {k:v for k,v in D.items() if len(v)>1}

In [ ]:
for i in D:
    
    hash_nova = hashlib.sha1(str(i).encode('utf-8')).hexdigest()    
    dados_tramitacao.id_tramitacao[D[i][1]:(D[i][1]+1)] = hash_nova

Dicionario politicos

In [ ]:
dicionario_deputados = {}
colunas = ["nome_completo", "nome"]

for i in dados_deputado.index:
    for j in colunas:
        dicionario_deputados[dados_deputado.loc[i, j]] = dados_deputado.id[i]

Indice invertido

In [ ]:
# preparando o corpus

import unicodedata
def remove_accents(input_str):
    if type(input_str) is float:
        if math.isnan(input_str):
            return 

    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

dados_tramitacao["despacho"] = dados_tramitacao["despacho"].apply(lambda x: remove_accents(x))
dados_tramitacao['despacho'] = dados_tramitacao['despacho'].apply(lambda texto: [token.lower() for token in WordPunctTokenizer().tokenize(texto)])

#pickle.dump(tramitacao, open('congresso_em_numeros/tramitacao.p', 'wb') )

In [ ]:
def default_to_dict(dic):
    for key in dic.keys():
        dic[key] = dict(dic[key])
    return dict(dic)

In [ ]:
import tqdm
indice = defaultdict(lambda: defaultdict(lambda: []))

for i, row in tqdm.tqdm(enumerate(dados_tramitacao.iterrows())):
    for j, token in enumerate(row[1]['despacho']):
        indice[token][row[0]].append(j)

        
#pickle.dump(default_to_dict(indice), open('congresso_em_numeros/indice.p', 'wb'))

Encontrar nome 

In [ ]:
def busca_frase(frase, indice):
        
    lista_frase = frase.split()
    
    #documeno conterá todos no inicio, para auxiliar na interseção
    documento = set(range(0, 10000000))
    
    #Objeto para receber as informações no final
    lista_documentos = {}
    
    #Vendo todos documentos que todas as palavras aparecerem
    for i in lista_frase:
        documento = documento.intersection(indice[i])
    
    #Passando pelos documentos
    for j in documento:
        
        id_despacho = dados_tramitacao.id_tramitacao[j]
        #Lista posição irá receber todas as posições que as palavras aparecem por documento
        lista_posicao = []
        #Passando por cada frase no loop de cada documento
        for i in lista_frase:
            
            lista_posicao.append(np.array(list(indice[i][j])))
        
        #Variável para auxiliar no for e podemos ter uma variável para receber as interseções
        aux_lista_posicao = lista_posicao[0]
        
        for h in range(1, len(lista_posicao)):
            
            #Somo 1 na lista auxiliar, porque pela ordenação a proxíma palavra tem que estar na posição seguinte 
            aux_lista_posicao = np.intersect1d(aux_lista_posicao +1, lista_posicao[h])                        
            
        #para não salvar os documentos vazios
        if len(aux_lista_posicao) > 0:
            lista_documentos[id_despacho] = (aux_lista_posicao - len(lista_posicao))
        
    return lista_documentos

In [ ]:
def todos_deputados(dic, indice):
    
    dic_final = defaultdict(lambda: list())
    
    for parlamentar in tqdm.tqdm(dic.keys(), total = len(dic.keys())):
        
        posicao = busca_frase(parlamentar, indice)
        
        if len(posicao) == 0:
            continue
        else:
            for pos in posicao.keys():
                dic_final[dic[parlamentar]].append(pos)
    
    return dic_final

In [ ]:
lista_final = todos_deputados(dicionario_deputados, indice)

In [ ]:
lista_final_2 = dict(lista_final)

In [ ]:
fileObject = open("congresso_em_numeros/lista_final",'wb') 
pickle.dump(lista_final_2 ,fileObject, protocol = 4)  
fileObject.close()

Criando a nossa lista de despachos e tokenizando

In [ ]:
dicionario_deputados

In [ ]:
lista_final[74173]

#### Analisis of voting data

In [ ]:
votacao_deputado.groupby(['voto']).count()['ano']

We can see that those are the types of votes on the database

- 'Sim' and 'Não' are Yes or No votes

- 'Obstrução' means Obstruction

- 'Abstenção' means Abstention

- 'Art. 17' means Article 17 which refers to prohibts the President of the House from voting

- 'Branco' means Blank vote, but it just appeared once, so it is certantly a mistake.

- '-' corresponds to the absence of the congressman

Since I am interested on congressman with the same behaviour that will form edges on the graph, it seems reasonable to delete the following vote types: 'Art. 17', 'Branco'.

#### Transforming '-' to 'Abstenção'

In [ ]:
votacao_deputado.voto[votacao_deputado.voto == '-'] = 'Abstenção'

In [ ]:
votacao_deputado = votacao_deputado[(votacao_deputado['voto'] != 'Art. 17') &
                 (votacao_deputado['voto'] != 'Branco')]
votacao_deputado.groupby(['voto']).count()['ano']

In [ ]:
votacao_deputado = votacao_deputado.drop(['codproposicao', 'tipo', 'numero', 'ano', 'codsessao',
       'idecadastro', 'partido', 'uf'], axis = 1)

In [ ]:
votacao_group = list(votacao_deputado.groupby(['votacao_id', 'voto']))

In [ ]:
base_final = votacao_group[0][1]
base_final = base_final.iloc[0:1]
base_final['edge'] = 'apagar'

In [ ]:
for i in range(0, len(votacao_group)):
    aux = votacao_group[i][1]
    
    for j in range(0, len(aux)):
        aux['edge'] = aux.nome[aux.index[j]]
        aux.edge[aux.index[j]] = 'apagar'
        
        base_final = base_final.append(aux)
        aux = aux.drop('edge', axis = 1)



In [ ]:
base_final